In [1]:
from seeq import spy #imports the spy connector to allow the connection to Seeq
import pandas as pd
from seeq.spy.assets import Asset
#To automatically run all lines of code, hit the "restart the kernel, then re-run the whole notebook (with dialog)" button (double right arrow)

# 1. Generate Workbook and Search Path

In [2]:
workbook_name = 'Brintnell NVA Report'#Edit Required-> change name to desired Seeq Workbook to access
signal_search_name = 'Liquid Flow Rate'#Edit Required-> enter the 1st signal required in your formula
search_path = 'Brintnell / Pelican Wells >> Producers >> * >> * >> *'# Edit Required-> Adjust based on Producers or Injector path you want the code to reference. * means ALL Assets in the corresponding hierarchy level

In [3]:
rate_result_df= spy.search(
    {"Name": signal_search_name, 
     "Path": search_path
    }, quiet = True).drop_duplicates(subset = ["Asset"], keep = "first")
#search all Producers (or Injectors if changed to "Injectors") in area for the signal Liquid Flow rate (signal_search_name)

In [4]:
def signal_search_check(search_result_df, signal_check_name):
    search_assets = search_result_df['Asset'].tolist()
    search_path = search_result_df['Path'].tolist()
    #Make a list of the asset & paths found for initail searched signals (signal_search_name)
    
    search_dict = {'Asset': search_assets} #generates a dictionary of all the Assets in search_results
    search_check_df = pd.DataFrame(search_dict)
    search_check_df['Name'] = '/^' + signal_check_name + '$/'
    search_check_df['Path'] = search_path
    return spy.search(search_check_df, quiet = True).drop_duplicates(subset = ["Asset"], keep = "first")
    #Search the list from "Liquid FLow Rates" signals and removes any duplicates (this step assists if there are multiple signals generated in the hiearchy due to an upload error, keeps the first signal only)

# 2. Create signal searches

In [5]:
rod_search_df = signal_search_check(search_result_df = rate_result_df , signal_check_name = 'Rod Speed')#Edit Required-> change the signal_check_name to the signal name required in your code. Update rod_search_df to the unique name required for your code
torque_search_df = signal_search_check(search_result_df = rate_result_df , signal_check_name = 'Torque')#Edit Required-> change the signal_check_name to the signal name required in your code. Update torque_search_df to the unique name required for your code
eff_search_df = signal_search_check(search_result_df = rate_result_df , signal_check_name = 'SAM_PMP_EFF')#Edit Required-> change the signal_check_name to the signal name required in your code. Update eff_search_df to the unique name required for your code

#need to list all of your signals here. Either Add/Delete rows if additional/fewer signals are required, respectively
#The signal_name_check 'variable' should be the exact name pulled from your Seeq workbook (ie. Rod Speed,JTF, Polymer Injection Rate etc.)


# 3. Create 1 Main Report

In [6]:
meta_data_df = rate_result_df.append(rod_search_df, ignore_index = True)#Edit Required-> update the "*_search_df" changed in Step #2
meta_data_df = meta_data_df.append(torque_search_df, ignore_index = True)#Edit Required-> update the "*_search_df" changed in Step #2
meta_data_df = meta_data_df.append(eff_search_df, ignore_index = True)#Edit Required-> update the "*_search_df" changed in Step #2
meta_data_df =  meta_data_df.assign(Field = meta_data_df['Path'].str.split(">>").str[2].str.strip())
meta_data_df =  meta_data_df.assign(Pad = meta_data_df['Path'].str.split(">>").str[3].str.strip())
meta_data_df['Build Path'] = 'NVA Variance Report'#Edit Required-> change 'NVA Variance Report' to desired Path Name for your code (this will be linked to WF2.)
meta_data_df['Build Asset'] = meta_data_df['Field']
#This step merges each individual signal report into 1 main report 

In [7]:
meta_data_df
#displays the main report

,ID,Path,Asset,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived,Field,Pad,Build Path,Build Asset
0,994C3934-A22F-4043-8D1B-55AB611DD659,Brintnell / Pelican Wells >> Producers >> Cent...,100/13-20-081-22W4,Liquid Flow Rate,BP01 100/13-20-081-22W4 Liquid Flow Rate,CalculatedSignal,NaN,Well Structure,False,Central Brintnell,BP01,NVA Variance Report,Central Brintnell
1,E558CF28-C46B-42AA-AA36-9DDC51D7E1C9,Brintnell / Pelican Wells >> Producers >> Cent...,103/05A-21-081-22W4,Liquid Flow Rate,BP02 103/05A-21-081-22W4 Liquid Flow Rate,CalculatedSignal,NaN,Well Structure,False,Central Brintnell,BP02,NVA Variance Report,Central Brintnell
2,E6E99BDE-03F7-4897-8C60-1E462D423E8F,Brintnell / Pelican Wells >> Producers >> Cent...,104/09D-20-081-22W4,Liquid Flow Rate,BP02 104/09D-20-081-22W4 Liquid Flow Rate,CalculatedSignal,NaN,Well Structure,False,Central Brintnell,BP02,NVA Variance Report,Central Brintnell
3,2230B928-BC4F-4CAE-87E3-9E8DE2BD003E,Brintnell / Pelican Wells >> Producers >> Cent...,102/12D-08-081-22W4,Liquid Flow Rate,BP03 102/12D-08-081-22W4 Liquid Flow Rate,CalculatedSignal,NaN,Well Structure,False,Central Brintnell,BP03,NVA Variance Report,Central Brintnell
4,168B79BC-863F-4D7C-A312-F611E8365B4A,Brintnell / Pelican Wells >> Producers >> Cent...,102/13C-08-081-22W4,Liquid Flow Rate,BP03 102/13C-08-081-22W4 Liquid Flow Rate,CalculatedSignal,NaN,Well Structure,False,Central Brintnell,BP03,NVA Variance Report,Central Brintnell
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4212,C15EAF3D-7449-49BC-8858-F8FE551C3EFB,Brintnell / Pelican Wells >> Producers >> Sout...,103/16D-07-081-22W4,SAM_PMP_EFF,WB42 103/16D-07-081-22W4 SAM_PMP_EFF,CalculatedSignal,NaN,Well Structure,False,South Brintnell,WB42,NVA Variance Report,South Brintnell
4213,29D7DF8F-874C-4FBE-A9B5-B3C73D3C7946,Brintnell / Pelican Wells >> Producers >> Sout...,100/03-32-080-22W4,SAM_PMP_EFF,WB43 100/03-32-080-22W4 SAM_PMP_EFF,CalculatedSignal,NaN,Well Structure,False,South Brintnell,WB43,NVA Variance Report,South Brintnell
4214,81E3BE1D-4044-41CA-BD39-8B81EF881862,Brintnell / Pelican Wells >> Producers >> Sout...,100/01-15-081-23W4,SAM_PMP_EFF,WB46 100/01-15-081-23W4 SAM_PMP_EFF,CalculatedSignal,NaN,Well Structure,False,South Brintnell,WB46,NVA Variance Report,South Brintnell
4215,7F179B90-9753-4849-9395-78395CD59D2A,Brintnell / Pelican Wells >> Producers >> Sout...,100/11D-01-081-23W4,SAM_PMP_EFF,WB46 100/11D-01-081-23W4 SAM_PMP_EFF,CalculatedSignal,NaN,Well Structure,False,South Brintnell,WB46,NVA Variance Report,South Brintnell


# 4. Create Formulas for workflow

In [8]:
#Copy/Paste all formulas from your Seeq Workbook (Seeq Workbook-> Variable-> Item Properties-> Formula)
#Assign a name to the formula (ie. variance_formula=...)
variance_formula = """
$twohr_formula= $d.movingAverageFilter(30min,2hr)
$twentyfourhr_formula= $d.movingAverageFilter(30min,24hr)
$frcond_20 = ((($twohr_formula-$twentyfourhr_formula)/$twentyfourhr_formula)*100).setUnits('%')
$frcond_20
"""

mvg_avg_formula = """
$frcond_20 = $v.movingAverageFilter(30min,2hr)
$frcond_20
"""

# 5. Create Defintions 

In [9]:
#definition or recipe for custom hierarchy
class Report(Asset):
    @Asset.Component()
    def Pads(self, metadata):
        return self.build_components(template = Pad, metadata = metadata, column_name = 'Pad')
                                     
class Pad(Asset):
    @Asset.Component()
    def Wells(self, metadata):
        return self.build_components(template = Well, metadata = metadata, column_name = 'Asset') 

class Well(Asset):
    @Asset.Attribute()
    def Liquid_Flow_Rate(self, metadata):#Edit Required-> Adjust name as required (ie. Liquid_Flow_Rate)
        return metadata[metadata['Name'].str.match('Liquid Flow Rate')]#Edit Required-> enter signal_search_name from Step #1

    @Asset.Attribute()
    def Flow_Rate_Variance(self, metadata):#Edit Required-> Adjust name as required (ie. Flow_Rate_Variance)
        return {
            'Type': 'Signal', #Edit Required-> Keep as Signal if it's a variable, change to condition if using a Tool Condition
            'Formula': variance_formula, #Edit Required-> Update to name for the formula used in Step #4
            'Formula Parameters': {
                '$d': self.Liquid_Flow_Rate(metadata)   #Edit Required-> ensure the $letter is same as Step #4            
            }           
            
        }

#Repeat the @ Liquid_Flow_Rate edits (just above) for the below def based on the signals/formulas your wanted to generate
    
    @Asset.Attribute()
    def Torque(self, metadata):
        return metadata[metadata['Name'].str.match('Torque')]

    @Asset.Attribute()
    def Torque_Variance(self, metadata):
        return {
            'Type': 'Signal',
            'Formula': variance_formula,
            'Formula Parameters': {
                '$d': self.Torque(metadata)                
            }           
            
        }
    @Asset.Attribute()
    def Pump_Eff(self, metadata):
        return metadata[metadata['Name'].str.match('SAM_PMP_EFF')]

    @Asset.Attribute()
    def Pump_Eff_Variance(self, metadata):
        return {
            'Type': 'Signal',
            'Formula': variance_formula,
            'Formula Parameters': {
                '$d': self.Pump_Eff(metadata)                
            }           
            
        }
    @Asset.Attribute()
    def Rod_Speed(self, metadata):
        return metadata[metadata['Name'].str.match('Rod Speed')]

    @Asset.Attribute()
    def Rod_Speed_Smoothed(self, metadata):
        return {
            'Type': 'Signal',
            'Formula': mvg_avg_formula,
            'Formula Parameters': {
                '$v': self.Rod_Speed(metadata)                
            }           
            
        }


In [10]:
#build asset tree from "Report" definition
build_df = spy.assets.build(Report, meta_data_df)

In [11]:
#remove assets with no type i.e wells without SAM_PUMP_EFF signal will be excluded
build_df = build_df[build_df['Type'].notna()].reset_index()

In [12]:
#check Build Result for unsucessful rows
build_df[build_df['Build Result'] != 'Success'][['Type', 'Asset', 'Path', 'Build Result']]

,Type,Asset,Path,Build Result


In [13]:
build_df[['ID','Type','Asset', 'Name', 'Build Result','Path', 'Asset Object', 'Template']]
#builds the report (items) that will be presented in your report

,ID,Type,Asset,Name,Build Result,Path,Asset Object,Template
0,NaN,Signal,100/13-20-081-22W4,Flow Rate Variance,Success,NVA Variance Report >> Central Brintnell >> BP01,NVA Variance Report >> Central Brintnell >> BP...,Well
1,994C3934-A22F-4043-8D1B-55AB611DD659,CalculatedSignal,100/13-20-081-22W4,Liquid Flow Rate,Success,NVA Variance Report >> Central Brintnell >> BP01,NVA Variance Report >> Central Brintnell >> BP...,Well
2,E616551E-394E-43C2-80C5-01191B3EB7DD,CalculatedSignal,100/13-20-081-22W4,Pump Eff,Success,NVA Variance Report >> Central Brintnell >> BP01,NVA Variance Report >> Central Brintnell >> BP...,Well
3,NaN,Signal,100/13-20-081-22W4,Pump Eff Variance,Success,NVA Variance Report >> Central Brintnell >> BP01,NVA Variance Report >> Central Brintnell >> BP...,Well
4,7DDD1658-7212-47EC-8799-F32EA19909B1,CalculatedSignal,100/13-20-081-22W4,Rod Speed,Success,NVA Variance Report >> Central Brintnell >> BP01,NVA Variance Report >> Central Brintnell >> BP...,Well
...,...,...,...,...,...,...,...,...
10573,1F9C5910-566C-4636-B044-B2C574747E54,CalculatedSignal,100/15C-01-081-23W4,Torque,Success,NVA Variance Report >> South Brintnell >> WB46,NVA Variance Report >> South Brintnell >> WB46...,Well
10574,NaN,Signal,100/15C-01-081-23W4,Torque Variance,Success,NVA Variance Report >> South Brintnell >> WB46,NVA Variance Report >> South Brintnell >> WB46...,Well
10575,NaN,Asset,100/15C-01-081-23W4,100/15C-01-081-23W4,Success,NVA Variance Report >> South Brintnell >> WB46,NVA Variance Report >> South Brintnell >> WB46...,Well
10576,NaN,Asset,WB46,WB46,Success,NVA Variance Report >> South Brintnell,NVA Variance Report >> South Brintnell >> WB46,Pad


In [16]:
#push asset tree to workbook defined by workbook_name variable assigned in Step #1
spy.push(metadata=build_df, workbook = workbook_name)

,index,Type,Formula,Formula Parameters,Name,Asset,Asset Object,Path,Template,Build Result,...,Field,Pad,Referenced Name,Referenced Path,Referenced Asset,Reference,Datasource Class,Datasource ID,Data ID,Push Result
0,0,CalculatedSignal,\n$twohr_formula= $d.movingAverageFilter(30min...,[d=9A80864B-5933-4939-ACD7-2F0DA4CBE5B0],Flow Rate Variance,100/13-20-081-22W4,NVA Variance Report >> Central Brintnell >> BP...,NVA Variance Report >> Central Brintnell >> BP01,Well,Success,...,NaN,NaN,NaN,NaN,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[768B1C69-AF81-4C3F-8229-0EB774A5B716] {Signal...,Success
1,1,CalculatedSignal,NaN,[signal=994C3934-A22F-4043-8D1B-55AB611DD659],Liquid Flow Rate,100/13-20-081-22W4,NVA Variance Report >> Central Brintnell >> BP...,NVA Variance Report >> Central Brintnell >> BP01,Well,Success,...,Central Brintnell,BP01,Liquid Flow Rate,Brintnell / Pelican Wells >> Producers >> Cent...,100/13-20-081-22W4,True,Seeq Data Lab,Seeq Data Lab,[768B1C69-AF81-4C3F-8229-0EB774A5B716] {Signal...,Success
2,2,CalculatedSignal,NaN,[signal=E616551E-394E-43C2-80C5-01191B3EB7DD],Pump Eff,100/13-20-081-22W4,NVA Variance Report >> Central Brintnell >> BP...,NVA Variance Report >> Central Brintnell >> BP01,Well,Success,...,Central Brintnell,BP01,SAM_PMP_EFF,Brintnell / Pelican Wells >> Producers >> Cent...,100/13-20-081-22W4,True,Seeq Data Lab,Seeq Data Lab,[768B1C69-AF81-4C3F-8229-0EB774A5B716] {Signal...,Success
3,3,CalculatedSignal,\n$twohr_formula= $d.movingAverageFilter(30min...,[d=33963A32-6E2C-4E6C-8DC4-D84727B5CB8C],Pump Eff Variance,100/13-20-081-22W4,NVA Variance Report >> Central Brintnell >> BP...,NVA Variance Report >> Central Brintnell >> BP01,Well,Success,...,NaN,NaN,NaN,NaN,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[768B1C69-AF81-4C3F-8229-0EB774A5B716] {Signal...,Success
4,4,CalculatedSignal,NaN,[signal=7DDD1658-7212-47EC-8799-F32EA19909B1],Rod Speed,100/13-20-081-22W4,NVA Variance Report >> Central Brintnell >> BP...,NVA Variance Report >> Central Brintnell >> BP01,Well,Success,...,Central Brintnell,BP01,Rod Speed,Brintnell / Pelican Wells >> Producers >> Cent...,100/13-20-081-22W4,True,Seeq Data Lab,Seeq Data Lab,[768B1C69-AF81-4C3F-8229-0EB774A5B716] {Signal...,Success
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10573,11200,CalculatedSignal,NaN,[signal=1F9C5910-566C-4636-B044-B2C574747E54],Torque,100/15C-01-081-23W4,NVA Variance Report >> South Brintnell >> WB46...,NVA Variance Report >> South Brintnell >> WB46,Well,Success,...,South Brintnell,WB46,Torque,Brintnell / Pelican Wells >> Producers >> Sout...,100/15C-01-081-23W4,True,Seeq Data Lab,Seeq Data Lab,[768B1C69-AF81-4C3F-8229-0EB774A5B716] {Signal...,Success
10574,11201,CalculatedSignal,\n$twohr_formula= $d.movingAverageFilter(30min...,[d=A5E176A3-4A48-4E26-9E58-AE209B3E2943],Torque Variance,100/15C-01-081-23W4,NVA Variance Report >> South Brintnell >> WB46...,NVA Variance Report >> South Brintnell >> WB46,Well,Success,...,NaN,NaN,NaN,NaN,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[768B1C69-AF81-4C3F-8229-0EB774A5B716] {Signal...,Success
10575,11202,Asset,NaN,NaN,100/15C-01-081-23W4,100/15C-01-081-23W4,NVA Variance Report >> South Brintnell >> WB46...,NVA Variance Report >> South Brintnell >> WB46,Well,Success,...,NaN,NaN,NaN,NaN,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[768B1C69-AF81-4C3F-8229-0EB774A5B716] {Asset}...,Success
10576,11203,Asset,NaN,NaN,WB46,WB46,NVA Variance Report >> South Brintnell >> WB46,NVA Variance Report >> South Brintnell,Pad,Success,...,NaN,NaN,NaN,NaN,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[768B1C69-AF81-4C3F-8229-0EB774A5B716] {Asset}...,Success
